<a href="https://colab.research.google.com/github/splendidcomputer/cora_project/blob/main/Mostafa_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/splendidcomputer/cora_project

Cloning into 'cora.content'...
fatal: repository 'https://github.com/splendidcomputer/cora_project/blob/main/cora.content/' not found


In [ ]:
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, optimizers, losses, metrics, Model
from keras.utils import np_utils
from tensorflow.keras.callbacks import Callback

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

In [ ]:
cora = pd.read_csv("./cora_project/cora.content", delimiter="\t", header=None)
y_labels = ["Case_Based", "Genetic_Algorithms", "Neural_Networks", "Probabilistic_Methods", "Reinforcement_Learning", "Rule_Learning", "Theory"]

for i,label in enumerate(y_labels):
    cora[1434][cora[1434]==label] = i

FileNotFoundError: ignored

In [ ]:
X = np.asarray(cora.iloc[:,1:-1]).astype('float32')
y = np.asarray(cora.iloc[:,-1]).astype('float32')

In [ ]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)


In [ ]:
model = Sequential()
model.add(layers.Dense(512, input_shape=(1433,), activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(7, activation='softmax'))
# compile the keras model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



In [ ]:
# Implement callback function to stop training
# when accuracy reaches e.g. ACCURACY_THRESHOLD = 0.95

VALIDATION_ACCURACY_THRESHOLD = 0.75

class myCallback(Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('val_accuracy') > VALIDATION_ACCURACY_THRESHOLD):   
          print("\nReached %2.2f%% accuracy, so stopping training!!" %(VALIDATION_ACCURACY_THRESHOLD*100))   
          self.model.stop_training = True

callbacks = myCallback()


In [ ]:
kFold = StratifiedKFold(n_splits=10)
# fit the keras model on the dataset
for train, test in kFold.split(X, encoded_Y):
  # convert integers to dummy variables (i.e. one hot encoded)
  dummy_y = np_utils.to_categorical(encoded_Y)
  model.fit(X[train], dummy_y[train], validation_data=(X[test], dummy_y[test]), epochs=20, batch_size=1, callbacks=[callbacks])

In [ ]:
list_predictions = []
folds_val_acc = []
for train, test in kFold.split(X, encoded_Y):
  fold_predictions = np.argmax(model.predict(X[test]), axis=1)
  list_predictions.append(fold_predictions) 
  dummy_y = np_utils.to_categorical(encoded_Y)
  folds_val_acc.append(model.evaluate(X[test], dummy_y[test]))

In [ ]:
folds_val_acc

In [ ]:
list_predictions

In [ ]:
final_predictions = []
for fold_prediction in list_predictions:
  for item in fold_prediction:
    final_predictions.append(y_labels[item])

In [ ]:
final_predictions_array = np.array([cora.iloc[:,0],final_predictions]).T

In [ ]:
final_predictions_array

In [ ]:
list_ids = list(cora.iloc[:,0])

In [ ]:
d = {'IDs': list_ids, 'Predictions': final_predictions}
df = pd.DataFrame(data=d)

In [ ]:
df.to_csv('final_predictions.tsv', sep="\t")